# Build Docker Image 
In this notebook we will build the docker container that contains the Resnet 152 model, Flask web application, model driver and all dependencies.

In [1]:
import os
from os import path
import json

In [2]:
!rm -r flaskwebapp

rm: cannot remove 'flaskwebapp': No such file or directory


In [3]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

In [4]:
!cp resnet_v1_152.ckpt flaskwebapp
!cp synset.txt flaskwebapp
!cp driver.py flaskwebapp
!ls flaskwebapp

driver.py  etc	nginx  resnet_v1_152.ckpt  synset.txt


Below is the module for the Flask web application.

In [5]:
%%writefile flaskwebapp/app.py
from flask import Flask, request
import time
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()


@app.route('/score', methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    predictions = predict_for(request_input)
    return json.dumps({'result': predictions})


@app.route("/")
def healthy():
    return "Healthy"


@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()


if __name__ == "__main__":
    app.run(host='0.0.0.0') # Ignore, Development server

Writing flaskwebapp/app.py


In [6]:
%%writefile flaskwebapp/wsgi.py
import sys
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


In [7]:
%%writefile flaskwebapp/requirements.txt
pillow
click==6.7
configparser==3.5.0
Flask==0.11.1
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

Writing flaskwebapp/requirements.txt


The configuration for the Nginx. Note that it creates a proxy between ports **80** and **5000**.

In [8]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


The image name below referes to our dockerhub account. If you wish to push the image to your account make sure you change this.

In [9]:
image_name = "masalvar/tfresnet-gpu"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [10]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [11]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal


def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()


Writing flaskwebapp/kill_supervisor.py


In [12]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 8 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [13]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy pandas scikit-learn && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
ENV PYTHONPATH /code/:$PYTHONPATH

RUN rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    pip install tensorflow-gpu==1.4.1 && \
    pip install -r /code/requirements.txt

EXPOSE 80
CMD ["supervisord", "-c", "/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


In [14]:
!docker build -t $image_name -f $docker_file_location $application_path

Sending build context to Docker daemon 241.6 MB
Step 1/16 : FROM nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
8.0-cudnn6-devel-ubuntu16.04: Pulling from nvidia/cuda

81ace0ea: Pulling fs layer 
3c87dba3: Pulling fs layer 
0a1c435a: Pulling fs layer 
4b14977e: Pulling fs layer 
96653dae: Pulling fs layer 
c8e60650: Pulling fs layer 
dd4a44b0: Pulling fs layer 
4b14977e: Waiting fs layer 
96653dae: Waiting fs layer 
d046ef8d: Waiting fs layer 
77e2a35a: Pull complete .3 MB/204.3 MBB1ADigest: sha256:9b6406c64c269429bd9bdfdf6a205524d0baaee25ada9b5f207654bfd3e8374b
Status: Downloaded newer image for nvidia/cuda:8.0-cudnn6-devel-ubuntu16.04
 ---> f100da99e951
Step 2/16 : MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>
 ---> Running in 24789dba59ab
 ---> dce18fde4669
Removing intermediate container 24789dba59ab
Step 3/16 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Running in f362

Get:11 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [77.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]
Get:13 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [602 kB]
Get:14 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]
Get:15 http://archive.ubuntu.com/ubuntu xenial/universe Sources [9802 kB]
Get:16 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
Get:17 http://security.ubuntu.com/ubuntu xenial-security/universe amd64 Packages [430 kB]
Get:18 http://security.ubuntu.com/ubuntu xenial-security/multiverse amd64 Packages [3489 B]
Get:19 http://archive.ubuntu.com/ubuntu xenial/main amd64 Packages [1558 kB]
Get:20 http://archive.ubuntu.com/ubuntu xenial/restricted amd64 Packages [14.1 kB]
Get:21 http://archive.ubuntu.com/ubuntu xenial/universe amd64 Packages [9827 kB]
Get:22 http://archive.ubuntu.com/ubuntu xenial/multiverse amd64 Packages [176 kB]
Get:23 h

Get:38 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjpeg-turbo8 amd64 1.4.2-0ubuntu3 [111 kB]
Get:39 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython2.7-minimal amd64 2.7.12-1ubuntu0~16.04.3 [340 kB]
Get:40 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python2.7-minimal amd64 2.7.12-1ubuntu0~16.04.3 [1261 kB]
Get:41 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python-minimal amd64 2.7.12-1~16.04 [28.1 kB]
Get:42 http://archive.ubuntu.com/ubuntu xenial/main amd64 mime-support all 3.59ubuntu1 [31.0 kB]
Get:43 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython2.7-stdlib amd64 2.7.12-1ubuntu0~16.04.3 [1880 kB]
Get:44 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python2.7 amd64 2.7.12-1ubuntu0~16.04.3 [224 kB]
Get:45 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libpython-stdlib amd64 2.7.12-1~16.04 [7768 B]
Get:46 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python amd64 2.7.1

Selecting previously unselected package libhcrypto4-heimdal:amd64.
Preparing to unpack .../libhcrypto4-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libhcrypto4-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libheimbase1-heimdal:amd64.
Preparing to unpack .../libheimbase1-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libheimbase1-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libwind0-heimdal:amd64.
Preparing to unpack .../libwind0-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libwind0-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected package libhx509-5-heimdal:amd64.
Preparing to unpack .../libhx509-5-heimdal_1.7~git20150920+dfsg-4ubuntu1.16.04.1_amd64.deb ...
Unpacking libhx509-5-heimdal:amd64 (1.7~git20150920+dfsg-4ubuntu1.16.04.1) ...
Selecting previously unselected packa

Unpacking git-man (1:2.7.4-0ubuntu1.3) ...
Selecting previously unselected package git.
Preparing to unpack .../git_1%3a2.7.4-0ubuntu1.3_amd64.deb ...
Unpacking git (1:2.7.4-0ubuntu1.3) ...
Selecting previously unselected package libfreetype6:amd64.
Preparing to unpack .../libfreetype6_2.6.1-0.1ubuntu2.3_amd64.deb ...
Unpacking libfreetype6:amd64 (2.6.1-0.1ubuntu2.3) ...
Selecting previously unselected package libfontconfig1:amd64.
Preparing to unpack .../libfontconfig1_2.11.94-0ubuntu1.1_amd64.deb ...
Unpacking libfontconfig1:amd64 (2.11.94-0ubuntu1.1) ...
Selecting previously unselected package libjpeg8:amd64.
Preparing to unpack .../libjpeg8_8c-2ubuntu8_amd64.deb ...
Unpacking libjpeg8:amd64 (8c-2ubuntu8) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../libtiff5_4.0.6-1ubuntu0.4_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.6-1ubuntu0.4) ...
Selecting previously unselected package libvpx3:amd64.
Preparing to unpack .../libvpx3_1.5.0-2ubuntu1_amd64.de

 ---> Running in 5de2a0c92204
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 55.6M  100 55.6M    0     0  58.3M      0 --:--:-- --:--:-- --:--:-- 58.2M
PREFIX=/opt/conda
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.0-h9df7e31_2 ...
installing: openssl-1.0.2n-hb7f436b_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.3-h55aa19d_2 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1-heed3624_0 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.22.0-h1bed415_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.1.

 ---> 093faa05986a
Removing intermediate container 5de2a0c92204
Step 11/16 : ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
 ---> Running in a97135dff534
 ---> ca527e4262ea
Removing intermediate container a97135dff534
Step 12/16 : ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
 ---> Running in 00d9ee264af6
 ---> 63bce52ccee7
Removing intermediate container 00d9ee264af6
Step 13/16 : ENV PYTHONPATH /code/:$PYTHONPATH
 ---> Running in 27104515a831
 ---> d07024841d97
Removing intermediate container 27104515a831
Step 14/16 : RUN rm /etc/nginx/sites-enabled/default &&     cp /code/nginx/app /etc/nginx/sites-available/ &&     ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ &&     pip install tensorflow-gpu==1.4.1 &&     pip install -r /code/requirements.txt
 ---> Running in 2c8e44b72c92
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in

Below we will push the image created to our dockerhub registry. Make sure you have already logged in to the appropriate dockerhub account using the docker login command

In [17]:
!docker push $image_name # If you haven't loged in to the approrpiate dockerhub account you will get an error

The push refers to a repository [docker.io/masalvar/tfresnet-gpu]

5ede532a: Preparing 
9173d41a: Preparing 
0de613b5: Preparing 
6e7461f1: Preparing 
09715d17: Preparing 
2e209605: Preparing 
ec85ab92: Preparing 
efe26028: Preparing 
a6197b2d: Preparing 
3d506f9c: Preparing 
67fc99a7: Preparing 
0ac0e989: Preparing 
75b88acb: Preparing 
4c622b50: Preparing 
ea2bb533: Preparing 
89ea437e: Preparing 
8b9b1b5b: Preparing 
0d5a7c40: Layer already exists latest: digest: sha256:a3d4672b9ec2b1d2fec3fa6d15bb75e966be414021a51b32216592ab8b4b2f71 size: 4091


In [18]:
print('Docker image name {}'.format(image_name)) 

Docker image name masalvar/tfresnet-gpu


### Test locally
Go to the [Test Locally notebook](03_TestLocally.ipynb) to test your Docker image